## C S 329E HW 9

# Crossvalidation and hyperparameter selection

## Your name here (and your partner's name if you are working in a pair)
Riley Sample
rcs3396

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [ ]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [ ]:
df_x, s_y = load_diabetes(return_X_y=True)
df_x = pd.DataFrame(df_x)
s_y = pd.DataFrame(s_y)
print(len(df_x))
print(len(s_y))

442
442


## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [ ]:
def get_linear_regression_model(df_x, s_y):
    temp = df_x.copy()
    temp[10] = 1
    # print(temp.shape, s_y.shape)
    x, residuals, rank, s = np.linalg.lstsq(temp, s_y)
    return x


In [ ]:
get_linear_regression_model(df_x, s_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


array([[ -10.01219782],
       [-239.81908937],
       [ 519.83978679],
       [ 324.39042769],
       [-792.18416163],
       [ 476.74583782],
       [ 101.04457032],
       [ 177.06417623],
       [ 751.27932109],
       [  67.62538639],
       [ 152.13348416]])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [ ]:
df_x

,0,1,2,3,4,5,6,7,8,9
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930


In [ ]:
def partition_data( df_x, s_y, k ):
    y_dict = {}
    x_dict = {}

    for i in range(1, k + 1):
        y_dict[i] = np.array([])
        x_dict[i] = np.array([[]])

    size = round(len(df_x) / k)
    for i in range(1, k + 1):
        if i != k:
            sample_x = df_x.sample(n=size)
            sample_y = s_y.sample(n=size)
        else:
            sample_x = df_x.sample(frac=1)
            sample_y = s_y.sample(frac=1)

        print(sample_x)

        df_x = df_x.drop(sample_x.index)
        s_y = s_y.drop(sample_y.index)

        x_dict[i] = sample_x
        y_dict[i] = sample_y

    return x_dict, y_dict

In [ ]:
(dict_k_df_x, dict_k_s_y) = partition_data( df_x, s_y, 5 )
for i in dict_k_s_y:
    print(len(dict_k_df_x[i]))

            0         1         2         3         4         5         6  \
274  0.016281 -0.044642 -0.024529  0.035644 -0.007073 -0.003193 -0.013948   
345  0.081666  0.050680 -0.002973 -0.033214  0.042462  0.057871 -0.010266   
436 -0.056370 -0.044642 -0.074108 -0.050428 -0.024960 -0.047034  0.092820   
69   0.016281 -0.044642 -0.046085 -0.005671 -0.075870 -0.061438 -0.013948   
94  -0.078165 -0.044642 -0.016984 -0.012556 -0.000193 -0.013527  0.070730   
..        ...       ...       ...       ...       ...       ...       ...   
330  0.009016  0.050680  0.030440  0.042530 -0.002945  0.036890 -0.065491   
212  0.067136 -0.044642  0.003494  0.035644  0.049341  0.031254  0.070730   
409 -0.052738  0.050680 -0.011595  0.056301  0.056221  0.072902 -0.039719   
186 -0.081798  0.050680  0.042296 -0.019442  0.039710  0.057558 -0.069172   
235  0.067136  0.050680 -0.014828  0.058596 -0.059359 -0.034508 -0.061809   

            7         8         9  
274 -0.002592  0.015567  0.015491  
345

## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [ ]:
def get_mae( s_y, s_y_hat):
    n = len(s_y)
    mae = sum(abs(s_y - s_y_hat))/n
    return mae

In [ ]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [ ]:
def get_train(y_dict, x_dict, k, fold):
    x_train = pd.DataFrame()
    y_train = pd.DataFrame()

    for i in range(1, k + 1):
        if i != fold:
            x_train = pd.concat([x_train, x_dict[i]], axis=0)
            y_train = np.append(y_train, y_dict[i])

    return x_train, y_train



In [ ]:
def get_y_hat(model, s_x):
    b = model[-1]
    x = np.delete(model, -1, 0)

    y_hat = np.matmul(s_x, x) + b
    
    return y_hat

In [ ]:
x_train, y_train = get_train(dict_k_s_y, dict_k_df_x, 5, 1)
print(x_train.shape, y_train.shape)

(354, 10) (354,)


In [ ]:
mae = np.array([])

for k in dict_k_df_x.keys():
    s_y = dict_k_s_y[k]
    s_x = dict_k_df_x[k]

    x_train, y_train = get_train(dict_k_s_y, dict_k_df_x, len(dict_k_df_x), k)

    model = get_linear_regression_model(x_train, y_train)

    s_y_hat = get_y_hat(model, s_x)
    
    mae = np.append( mae, get_mae(s_y,s_y_hat) )  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


In [ ]:
print('mean:', mae.mean())
print('min:', mae.min())
print('min:', mae.max())

mean: 220.624797979798
min: 193.04444444444445
min: 243.07954545454547


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [ ]:
df_x, s_y = load_iris(return_X_y=True)
df_x = pd.DataFrame(df_x)
s_y = pd.DataFrame(s_y)

In [ ]:
df_x

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
s_y

,0
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [ ]:
(dict_k_df_x, dict_k_s_y) = partition_data( df_x, s_y, 5 )

       0    1    2    3
128  6.4  2.8  5.6  2.1
47   4.6  3.2  1.4  0.2
148  6.2  3.4  5.4  2.3
86   6.7  3.1  4.7  1.5
136  6.3  3.4  5.6  2.4
20   5.4  3.4  1.7  0.2
140  6.7  3.1  5.6  2.4
147  6.5  3.0  5.2  2.0
108  6.7  2.5  5.8  1.8
94   5.6  2.7  4.2  1.3
134  6.1  2.6  5.6  1.4
14   5.8  4.0  1.2  0.2
33   5.5  4.2  1.4  0.2
79   5.7  2.6  3.5  1.0
19   5.1  3.8  1.5  0.3
91   6.1  3.0  4.6  1.4
85   6.0  3.4  4.5  1.6
29   4.7  3.2  1.6  0.2
132  6.4  2.8  5.6  2.2
72   6.3  2.5  4.9  1.5
110  6.5  3.2  5.1  2.0
74   6.4  2.9  4.3  1.3
55   5.7  2.8  4.5  1.3
5    5.4  3.9  1.7  0.4
100  6.3  3.3  6.0  2.5
75   6.6  3.0  4.4  1.4
115  6.4  3.2  5.3  2.3
80   5.5  2.4  3.8  1.1
40   5.0  3.5  1.3  0.3
76   6.8  2.8  4.8  1.4
       0    1    2    3
141  6.9  3.1  5.1  2.3
120  6.9  3.2  5.7  2.3
17   5.1  3.5  1.4  0.3
9    4.9  3.1  1.5  0.1
93   5.0  2.3  3.3  1.0
25   5.0  3.0  1.6  0.2
6    4.6  3.4  1.4  0.3
42   4.4  3.2  1.3  0.2
2    4.7  3.2  1.3  0.2
58   6.6  2.9  4

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [ ]:
def get_acc( s_1, s_2 ):
    s_1 = s_1[0]
    count = np.sum(s_1 == s_2)
    acc = count/len(s_1)
    return acc


In [ ]:
get_acc(s_y, np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [ ]:
def get_folds(x_dict, y_dict, k):
  x_dict.pop(k)
  y_dict.pop(k)

  return x_dict, y_dict

In [ ]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
this_acc = 0
outer_acc = np.array([])
for k in dict_k_df_x.keys():
    
    best_acc = 0
    best_min_impurity = 0
    for pos_min_impurity in possible_min_impurity_decrease:
        
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        acc = 0
        for k in dict_k_df_x.keys():
            x_train, y_train = get_train(dict_k_s_y, dict_k_df_x, 5, k)
            model = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=pos_min_impurity)
            model = model.fit(x_train, y_train)
            y_pred = model.predict(dict_k_df_x[k])
            acc += get_acc(dict_k_s_y[k], y_pred)

        mean_acc = acc/4
        print('Testing', pos_min_impurity, 'min impurity decrease')
        print('   Average accuracy over 4 folds is', mean_acc)

        if mean_acc > best_acc:
            best_acc = mean_acc
            best_min_impurity = pos_min_impurity

    print('\n Best min impurity decrease is', best_min_impurity, '\n')

    # Use best min impurity decrease to train model
    x_train, y_train = get_train(dict_k_s_y, dict_k_df_x, 5, k)
    model = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=best_min_impurity)
    model = model.fit(x_train, y_train)
    y_pred = model.predict(dict_k_df_x[k])
    
    # outer accuracy calculation 
    this_acc = get_acc(dict_k_s_y[k], y_pred)
    outer_acc = np.append(outer_acc, this_acc) # make sure and calculate this_acc in your loop

Testing 0.1 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.25 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.3 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.4 min impurity decrease
   Average accuracy over 4 folds is 0.3

 Best min impurity decrease is 0.1 

Testing 0.1 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.25 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.3 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.4 min impurity decrease
   Average accuracy over 4 folds is 0.3

 Best min impurity decrease is 0.1 

Testing 0.1 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.25 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.3 min impurity decrease
   Average accuracy over 4 folds is 0.3
Testing 0.4 min impurity decrease
   Average accuracy over 4 folds is 0.3

 Best min impurity decrease is 0

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [ ]:
print('min:', outer_acc.min())
print('max:', outer_acc.max())
print('mean:', outer_acc.mean())

min: 0.26666666666666666
max: 0.26666666666666666
mean: 0.26666666666666666
